In [167]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from afinn import Afinn
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')
from datetime import datetime


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ange/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/ange/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ange/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [168]:
df = pd.read_csv("news_data/news_data_preprocessed.csv")

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   pub_date         1460 non-null   object
 1   headline.main    1460 non-null   object
 2   snippet          1460 non-null   object
 3   headline_lemmas  1460 non-null   object
 4   snippet_lemmas   1460 non-null   object
dtypes: object(5)
memory usage: 57.2+ KB


In [170]:
df.head()

,pub_date,headline.main,snippet,headline_lemmas,snippet_lemmas
0,2016-01-01,No More Statutes of Limitations for Rape Fredd...,Bill Cosby came close to escaping sexual assau...,"['statute', 'limitation', 'rape', 'freddie', '...","['bill', 'cosby', 'came', 'close', 'escaping',..."
1,2016-01-02,Wishes for the New Year A Smarter Plan to Make...,"From Times Square to the web, we asked people ...","['wish', 'new', 'year', 'smarter', 'plan', 'ma...","['time', 'square', 'web', 'asked', 'people', '..."
2,2016-01-03,"For ‘Game of Thrones’ Readers, New HBO Season ...",The author George R.R. Martin said he had miss...,"['game', 'throne', 'reader', 'new', 'hbo', 'se...","['author', 'george', 'martin', 'said', 'missed..."
3,2016-01-04,"Auto Sales, the Fed’s December Meeting and Emp...","This week, automakers are expected to post big...","['auto', 'sale', 'fed', 'december', 'meeting',...","['week', 'automaker', 'expected', 'post', 'big..."
4,2016-01-05,Saudi Arabia’s Dangerous Sectarian Game G.M. C...,The Saudi royals think that stoking hatred of ...,"['saudi', 'arabia', 'dangerous', 'sectarian', ...","['saudi', 'royal', 'think', 'stoking', 'hatred..."


# VADER

In [171]:
sia = SentimentIntensityAnalyzer()

In [172]:
df['snippet_VADER'] = df['snippet'].astype(str).apply(lambda x: sia.polarity_scores(x)['compound'])

In [173]:
df['headline_VADER'] = df['headline.main'].astype(str).apply(lambda x: sia.polarity_scores(x)['compound'])

# AFINN

In [174]:
afinn = Afinn()
df["snippet_AFINN"] = df["snippet_lemmas"].explode().apply(lambda word: afinn.score(word)).groupby(level=0).sum()
df["headline_AFINN"] = df["headline_lemmas"].explode().apply(lambda word: afinn.score(word)).groupby(level=0).sum()

In [177]:
df.head()

,pub_date,headline.main,snippet,headline_lemmas,snippet_lemmas,snippet_VADER,headline_VADER,snippet_AFINN,headline_AFINN
0,2016-01-01,No More Statutes of Limitations for Rape Fredd...,Bill Cosby came close to escaping sexual assau...,"['statute', 'limitation', 'rape', 'freddie', '...","['bill', 'cosby', 'came', 'close', 'escaping',...",-0.9113,-0.9945,3.0,-48.0
1,2016-01-02,Wishes for the New Year A Smarter Plan to Make...,"From Times Square to the web, we asked people ...","['wish', 'new', 'year', 'smarter', 'plan', 'ma...","['time', 'square', 'web', 'asked', 'people', '...",0.9970,0.9920,21.0,14.0
2,2016-01-03,"For ‘Game of Thrones’ Readers, New HBO Season ...",The author George R.R. Martin said he had miss...,"['game', 'throne', 'reader', 'new', 'hbo', 'se...","['author', 'george', 'martin', 'said', 'missed...",-0.9638,0.9703,-16.0,-17.0
3,2016-01-04,"Auto Sales, the Fed’s December Meeting and Emp...","This week, automakers are expected to post big...","['auto', 'sale', 'fed', 'december', 'meeting',...","['week', 'automaker', 'expected', 'post', 'big...",0.9966,-0.9847,32.0,-1.0
4,2016-01-05,Saudi Arabia’s Dangerous Sectarian Game G.M. C...,The Saudi royals think that stoking hatred of ...,"['saudi', 'arabia', 'dangerous', 'sectarian', ...","['saudi', 'royal', 'think', 'stoking', 'hatred...",-0.6580,-0.9988,-30.0,-95.0


In [178]:
df.describe()

,snippet_VADER,headline_VADER,snippet_AFINN,headline_AFINN
count,1460.000000,1460.000000,1460.000000,1460.000000
mean,0.357351,-0.595909,-7.901370,-35.249315
std,0.894007,0.750961,42.704123,32.456490
min,-0.999900,-1.000000,-278.000000,-399.000000
25%,-0.932850,-0.996700,-35.000000,-54.000000
50%,0.988400,-0.989300,-8.000000,-33.000000
75%,0.998200,-0.876425,17.000000,-15.000000
max,0.999900,0.999400,156.000000,103.000000
